In [33]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from python_module.pricing_model import BlackScholesModel, SABRModel

In [34]:
# -------
# CUSTOM FUNCTION
# -------
def compute_vol_bump(spot_change, implied_vol, spot_vol_correl, vol_of_vol):

    vol_change = (vol_of_vol / implied_vol) * spot_vol_correl * spot_change
    vol_bump = implied_vol * vol_change
    
    return vol_bump
    
def compute_slide_pnl(market_data_df, slide_list, slide_type='spot-vol'):
    results = list()

    T = market_data_df['T'].unique()[0]
    est_alpha, est_rho, est_nu = SABRModel.solve_parameters(F=F, T=T, strikes=market_data_df['K'].to_list(), market_vols=market_data_df['IV'].to_list())

    for slide in slide_list:
        
        for index in market_data_df.index:
            S0, K, T, r, IV, option_type, delta = market_data_df.loc[index, ['S0', 'K', 'T', 'r', 'IV', 'option_type', 'delta']]
            S0_bumped = S0 * (1+slide)
            delta_hedge_pnl = S0*delta*slide*-1
            
            if slide_type == 'spot-only':
                results.append({'slide': f'slide spot-only {slide:.2f}', 'index': index, 'delta_hedge_pnl': delta_hedge_pnl, 'price': BlackScholesModel.compute_option(S0_bumped, K, T, r, IV, option_type, False)})
            
            elif slide_type == 'spot-vol':
                vol_bump = compute_vol_bump(slide, IV, est_rho, est_nu)
                IV_bumped = IV + vol_bump
                results.append({'slide': f'slide spot-vol {slide:.2f}', 'index': index, 'delta_hedge_pnl': delta_hedge_pnl, 'price': BlackScholesModel.compute_option(S0_bumped, K, T, r, IV_bumped, option_type, False)})
            else:
                raise ValueError
    slide_pnl = pd.DataFrame(results).pivot_table(index='index', columns='slide', values='price').add(-market_data_df['price'], axis=0)
    delta_hedge_pnl = pd.DataFrame(results).pivot_table(index='index', columns='slide', values='delta_hedge_pnl')
    total_pnl = slide_pnl + delta_hedge_pnl

    return total_pnl.loc[market_data_df.index]

In [35]:
# -------
# GENERATE MARKET DATA AND GREEKS
# -------

# Explicit pricing parameters
S0 = F = 100
r = 0
option_type = 'put'
time_to_maturity = 250

# Market parameters
market_alpha = +0.1
market_beta  = +1.0
market_rho   = -0.5
market_nu    = 1.0

# Realized parameters
realized_alpha = +0.1
realized_beta  = +1.0
realized_rho   = -0.7
realized_nu    = 1.5

# Pre-processing
T = time_to_maturity / 250

# Pricing factory
market_data_list = list()
realized_data_list = list()
for K in np.linspace(start=80, stop=120, num=20, dtype=int):

    option_type = 'call' if K > 100 else 'put'
    
    market_pricing_results = SABRModel.compute_option(F, K, T, market_alpha, market_beta, market_rho, market_nu, r, option_type)
    realized_pricing_results = SABRModel.compute_option(F, K, T, realized_alpha, realized_beta, realized_rho, realized_nu, r, option_type)
    
    market_data_list.append({
        'symbol': f"{time_to_maturity}_{K}_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': time_to_maturity, 
        'S0':S0, 'r': r, 'F': F, 'K': K, 'T': T, **market_pricing_results})

    realized_data_list.append({
        'symbol': f"{time_to_maturity}_{K}_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': time_to_maturity, 
        'S0':S0, 'r': r, 'F': F, 'K': K, 'T': T, **realized_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

realized_data_df = pd.DataFrame(realized_data_list)
realized_data_df = realized_data_df.set_index('symbol')

In [68]:
slide_list = [-0.05, 0.05]
slide_df = compute_slide_pnl(market_data_df, slide_list, slide_type='spot-only')
expected_pnl = market_data_df['price']-realized_data_df['price']

In [69]:
slide_df.head()

slide,slide spot-only -0.05,slide spot-only 0.05
symbol,,
250_80_put,0.130654,0.097921
250_82_put,0.153507,0.115731
250_84_put,0.180153,0.136886
250_86_put,0.211013,0.161978
250_88_put,0.246372,0.191637


In [71]:
c = -expected_pnl.values
A_ub = slide_df.transpose().to_numpy()
b_ub = np.array([0]*slide_df.shape[1])

bounds = [(-100, 100)] * slide_df.shape[0]

res = linprog(
    c=c,
    A_ub=A_ub, 
    b_ub=b_ub, 
    bounds=bounds,
    method="highs"
)
slide_df['weights'] = res.x

In [72]:
slide_df

slide,slide spot-only -0.05,slide spot-only 0.05,weights
symbol,,,
250_80_put,0.130654,0.097921,-100.000000
250_82_put,0.153507,0.115731,-100.000000
250_84_put,0.180153,0.136886,-100.000000
250_86_put,0.211013,0.161978,-100.000000
250_88_put,0.246372,0.191637,-100.000000
250_90_put,0.286211,0.226447,-100.000000
250_92_put,0.329907,0.266789,-100.000000
250_94_put,0.375786,0.312519,-100.000000
250_96_put,0.420498,0.362404,-100.000000


In [60]:
slide_df

slide,slide spot-only -0.05,slide spot-only 0.05,PnL,weights
symbol,,,,
250_80_put,0.130654,0.097921,-0.788334,-100.000000
250_82_put,0.153507,0.115731,-0.788505,-100.000000
250_84_put,0.180153,0.136886,-0.779731,-100.000000
250_86_put,0.211013,0.161978,-0.759375,-100.000000
250_88_put,0.246372,0.191637,-0.724069,-100.000000
250_90_put,0.286211,0.226447,-0.669547,-100.000000
250_92_put,0.329907,0.266789,-0.590564,-100.000000
250_94_put,0.375786,0.312519,-0.481076,-100.000000
250_96_put,0.420498,0.362404,-0.335253,-100.000000
